# Imports

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

# Layers

In [3]:
# Network with one fully connected hidden layer with 256 units and ReLU activation, followed by a fully connected output layer with ten units (no activation function).
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

# Creating Neural Network Modules

## Custom Module

In [4]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

torch.Size([2, 10])

## Sequential Module

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In [7]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

## Modules with constant parameters and control flow

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
model = FixedHiddenMLP()
model(X)

tensor(0.1802, grad_fn=<SumBackward0>)

# Parameter Management

## Parameter Access

In [10]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0775,  0.0329, -0.1264,  0.1548,  0.2236,  0.1800,  0.1299,  0.0654,
                        0.0100,  0.2154,  0.2110,  0.1630, -0.0928, -0.0936,  0.0539,  0.0765,
                       -0.0682,  0.0993,  0.0199, -0.0616],
                      [-0.1443,  0.2143,  0.1502, -0.1033,  0.1659,  0.2139,  0.0898, -0.0140,
                        0.0904, -0.0145,  0.0308, -0.1710, -0.1666,  0.0924, -0.0421, -0.0837,
                        0.1154, -0.1473, -0.2033, -0.1613],
                      [ 0.1085, -0.0482,  0.0457,  0.2106,  0.1395,  0.0422, -0.1919, -0.0496,
                        0.1927, -0.0873,  0.0009,  0.1187, -0.2014,  0.0671, -0.0275,  0.2159,
                        0.0507,  0.0360,  0.2112, -0.2059],
                      [-0.1645,  0.0238, -0.0547,  0.1689,  0.0775, -0.2104, -0.0786,  0.0267,
                       -0.0847, -0.1531,  0.0583,  0.1872, -0.1668,  0.1951,  0.1574,  0.1777,
                        0.193

In [11]:
model.linear.bias

Parameter containing:
tensor([ 0.0794,  0.0911,  0.2195, -0.1683, -0.0377,  0.0771, -0.2058, -0.1873,
        -0.0950, -0.0177, -0.0140,  0.0674,  0.0539, -0.0869,  0.0280, -0.1657,
         0.1887,  0.0182, -0.0703,  0.0382], requires_grad=True)

In [12]:
model.linear.weight

Parameter containing:
tensor([[-0.0775,  0.0329, -0.1264,  0.1548,  0.2236,  0.1800,  0.1299,  0.0654,
          0.0100,  0.2154,  0.2110,  0.1630, -0.0928, -0.0936,  0.0539,  0.0765,
         -0.0682,  0.0993,  0.0199, -0.0616],
        [-0.1443,  0.2143,  0.1502, -0.1033,  0.1659,  0.2139,  0.0898, -0.0140,
          0.0904, -0.0145,  0.0308, -0.1710, -0.1666,  0.0924, -0.0421, -0.0837,
          0.1154, -0.1473, -0.2033, -0.1613],
        [ 0.1085, -0.0482,  0.0457,  0.2106,  0.1395,  0.0422, -0.1919, -0.0496,
          0.1927, -0.0873,  0.0009,  0.1187, -0.2014,  0.0671, -0.0275,  0.2159,
          0.0507,  0.0360,  0.2112, -0.2059],
        [-0.1645,  0.0238, -0.0547,  0.1689,  0.0775, -0.2104, -0.0786,  0.0267,
         -0.0847, -0.1531,  0.0583,  0.1872, -0.1668,  0.1951,  0.1574,  0.1777,
          0.1939, -0.0901,  0.0100,  0.0737],
        [-0.1718, -0.0259, -0.1604, -0.0556,  0.0542,  0.0288, -0.1707,  0.2051,
         -0.0273, -0.0339,  0.1391, -0.1435,  0.0386, -0.1638, -0

In [13]:
[(name, param.shape) for name, param in model.named_parameters()]

[('linear.weight', torch.Size([20, 20])), ('linear.bias', torch.Size([20]))]

## Tied Parameters

In [14]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# Parameter Initialization

## Built-in Initialization

In [ ]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0009, -0.0068, -0.0071, -0.0107, -0.0018,  0.0019, -0.0017, -0.0086,
         -0.0058, -0.0020, -0.0180,  0.0125,  0.0091,  0.0035, -0.0111, -0.0028,
         -0.0041,  0.0070,  0.0023,  0.0064]),
 tensor(0.))

In [16]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]),
 tensor(0.))

In [17]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4260,  0.2383,  0.0391,  0.0382, -0.3253, -0.3273,  0.3385,  0.3767,
        -0.2979,  0.3159, -0.0145, -0.0432,  0.4061, -0.2579,  0.4494,  0.2478,
         0.1121,  0.0312, -0.3097, -0.1953])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.]])


## Custom Initialization

In [18]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 20])
Init weight torch.Size([8, 8])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  9.2186,  0.0000, -0.0000, -8.2066, -0.0000, -0.0000, -7.0023,
         -6.6691,  0.0000, -0.0000,  0.0000,  0.0000,  9.9865,  6.0860, -0.0000,
         -0.0000, -0.0000,  5.9988, -6.2775],
        [ 5.5659, -0.0000,  9.3292,  0.0000,  6.7719,  5.8093, -0.0000,  7.0174,
          7.5207,  0.0000,  0.0000, -0.0000,  7.3183,  0.0000, -0.0000,  7.6960,
          0.0000,  0.0000,  8.9633, -0.0000]], grad_fn=<SliceBackward0>)

In [19]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.2186,  1.0000,  1.0000, -7.2066,  1.0000,  1.0000, -6.0023,
        -5.6691,  1.0000,  1.0000,  1.0000,  1.0000, 10.9865,  7.0860,  1.0000,
         1.0000,  1.0000,  6.9988, -5.2775])

# Lazy Initialization

For cases in which we do not define the dimensions of the layers of the Neural Network beforehand and instead initialize it based on the input dimensions

In [24]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

In [25]:
net[0].weight


<UninitializedParameter>

In [27]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

# Custom Layers

Layers Without Parameters

In [28]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [29]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [30]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

In [31]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

## Layers With Parameters

In [32]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [33]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.0904, -1.7410,  0.1128],
        [ 0.6905,  0.0621,  0.4388],
        [ 0.1429, -1.2900,  0.1187],
        [-0.4359,  1.7918,  0.3152],
        [-2.7833,  1.2190,  0.5276]], requires_grad=True)

In [34]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 1.2237],
        [0.0000, 0.1485, 1.2916]])

In [35]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

# File I/O

## Loading and Saving Tensors

In [36]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [37]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [38]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [39]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## Saving and Loading Model Parameters

In [40]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [41]:
torch.save(net.state_dict(), 'mlp.params')

In [42]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [43]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# Utilizing the GPU

## Computing Devices

In [44]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [45]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

1

In [46]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## Tensors and GPUs

In [47]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

### Storing it in the GPU

In [51]:
#X = torch.ones(2, 3, device=try_gpu())
#X

In [ ]:
#Z = X.cuda(1)
#print(X)
#print(Z)

In [52]:
#Z.cuda(1) is Z

# Neural Networks and GPUs

In [55]:
"""net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())
net(X)
net[0].weight.data.device"""

'net = nn.Sequential(nn.LazyLinear(1))\nnet = net.to(device=try_gpu())\nnet(X)\nnet[0].weight.data.device'

In [54]:
"""def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model"""

'def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):\n    self.save_hyperparameters()\n    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]\n\ndef prepare_batch(self, batch):\n    if self.gpus:\n        batch = [a.to(self.gpus[0]) for a in batch]\n    return batch\n\n@d2l.add_to_class(d2l.Trainer)  #@save\ndef prepare_model(self, model):\n    model.trainer = self\n    model.board.xlim = [0, self.max_epochs]\n    if self.gpus:\n        model.to(self.gpus[0])\n    self.model = model'